In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor


In [2]:
data = pd.read_csv('CC2020_train_final.csv')

In [3]:
data.head(3)

,YEAR,LOCATION,INBRED,INBRED_CLUSTER,TESTER,TESTER_CLUSTER,YIELD
0,18,Loc 5608,Inbred_142,Cluster11,Tester_740,Cluster10,1.135462
1,18,Loc 4533,Inbred_142,Cluster11,Tester_740,Cluster10,1.139813
2,18,Loc 5620,Inbred_142,Cluster11,Tester_740,Cluster10,1.117778


In [4]:
X = data.drop(['YEAR', 'LOCATION'], axis=1)

In [5]:
len(X)

199476

In [6]:
clean_data = X.groupby(['INBRED', 'INBRED_CLUSTER','TESTER', 'TESTER_CLUSTER' ],as_index=False).mean()

In [7]:
clean_data.head(5)

,INBRED,INBRED_CLUSTER,TESTER,TESTER_CLUSTER,YIELD
0,Inbred_1071,Cluster8,Tester_1345,Cluster1,0.986544
1,Inbred_1071,Cluster8,Tester_4373,Cluster3,1.057704
2,Inbred_1071,Cluster8,Tester_4473,Cluster5,1.023704
3,Inbred_1071,Cluster8,Tester_4541,Cluster4,1.014735
4,Inbred_1071,Cluster8,Tester_5305,Cluster4,1.062727


In [8]:
len(clean_data)

10919

In [9]:
y = clean_data.YIELD

In [10]:
data = clean_data.drop(['YIELD'], axis=1)

In [11]:
data.head(5)

,INBRED,INBRED_CLUSTER,TESTER,TESTER_CLUSTER
0,Inbred_1071,Cluster8,Tester_1345,Cluster1
1,Inbred_1071,Cluster8,Tester_4373,Cluster3
2,Inbred_1071,Cluster8,Tester_4473,Cluster5
3,Inbred_1071,Cluster8,Tester_4541,Cluster4
4,Inbred_1071,Cluster8,Tester_5305,Cluster4


In [12]:
X = data

In [13]:
label_enc = preprocessing.LabelEncoder()

In [14]:
X['INBRED'] = label_enc.fit_transform(X['INBRED'])
X['INBRED_CLUSTER'] = label_enc.fit_transform(X['INBRED_CLUSTER'])
X['TESTER'] = label_enc.fit_transform(X['TESTER'])
X['TESTER_CLUSTER'] = label_enc.fit_transform(X['TESTER_CLUSTER'])

In [15]:
X.head(3)

,INBRED,INBRED_CLUSTER,TESTER,TESTER_CLUSTER
0,0,12,0,0
1,0,12,125,7
2,0,12,136,9


In [16]:
#notice how these are all same.. number varies within same location or year

In [17]:
enc = preprocessing.OneHotEncoder()

In [18]:
X = enc.fit_transform(X)

#Classifiers

In [19]:
#GradientBoosting Basic

In [20]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

loss = ['ls', 'lad', 'huber', 'quantile']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

subsample = [0.8,1,1.2]

min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'loss': loss,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'subsample': subsample}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'loss': ['ls', 'lad', 'huber', 'quantile'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'subsample': [0.8, 1, 1.2]}


In [22]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
gb = GradientBoostingRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
gb_random = RandomizedSearchCV(estimator = gb, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
gb_random.fit(X, y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  6.1min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 21.6min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 50.2min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=3,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                                       min_weight_fraction_leaf=0.0,
                             

In [24]:
gb_random.best_params_

{'subsample': 0.8,
 'n_estimators': 800,
 'min_samples_split': 2,
 'min_samples_leaf': 4,
 'max_depth': 10,
 'loss': 'ls'}

In [36]:
scores = []

for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=np.random)
    my_model = GradientBoostingRegressor(subsample = 0.8, n_estimators=1000, learning_rate=0.05,
                                   max_depth=10, max_features='sqrt',
                                   min_samples_leaf=4, min_samples_split=2, 
                                   loss='ls', random_state =5)
    my_model.fit(X_train, y_train)
    predictions = my_model.predict(X_test)
    score = mean_absolute_error(predictions, y_test)
    scores.append(score)

    
print('Total CV MSE is {}'.format(np.mean(scores)))
print('Total STD is {}'.format(np.std(scores)))

Total CV MSE is 0.039971283609207965
Total STD is 0.00035965407842892963


In [44]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'n_estimators': [1000,800],
               'loss': ['ls'],
               'max_depth': [9,10,13],
               'min_samples_split': [1,2,4],
               'min_samples_leaf': [3,4,6],
               'subsample': [0.7,0.8,1.0]
}


In [45]:
# Create a based model
gb = GradientBoostingRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = gb, param_grid = param_grid, 
                          cv = 3, verbose = 2, n_jobs = -1)

In [46]:
# Fit the grid search to the data
grid_search.fit(X, y)

Fitting 3 folds for each of 162 candidates, totalling 486 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   11.8s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 486 out of 486 | elapsed:  6.7min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                 criterion='friedman_mse',
                                                 init=None, learning_rate=0.1,
                                                 loss='ls', max_depth=3,
                                                 max_features=None,
                                                 max_leaf_nodes=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=100,
                                                 n_iter_n...
                            

In [47]:
grid_search.best_params_

{'loss': 'ls',
 'max_depth': 9,
 'min_samples_leaf': 3,
 'min_samples_split': 2,
 'n_estimators': 800,
 'subsample': 0.7}

In [53]:
scores = []

for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=np.random)
    my_model = GradientBoostingRegressor(subsample = 0.7, n_estimators=800, learning_rate=0.05,
                                   max_depth=9, max_features='sqrt',
                                   min_samples_leaf=2, min_samples_split=2, 
                                   loss='ls', random_state =5)
    my_model.fit(X_train, y_train)
    predictions = my_model.predict(X_test)
    score = mean_absolute_error(predictions, y_test)
    scores.append(score)

    
print('Total CV MSE is {}'.format(np.mean(scores)))
print('Total STD is {}'.format(np.std(scores)))

Total CV MSE is 0.03995985956471137
Total STD is 0.0004024784307057446
